## **Federated Learning for attack classification: 7 nodes sharing gradients**

IDs from this file = **idcat7xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(Dense(108, input_shape=input_shape, activation='relu'))
    model.add(layers.Conv2D(filters=16, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(108, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['dos', 'reconnaissance', 'generic', 'exploits'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

### Cat7A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7A-Part7.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [6]:

num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idCAT700.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [12]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT700.hdf5')

Epoch 1/5
22/22 [==============================] - 2s 80ms/step - loss: 0.5719 - accuracy: 0.9876 - val_loss: 1.4407 - val_accuracy: 0.1422
Epoch 2/5
22/22 [==============================] - 2s 71ms/step - loss: 0.1428 - accuracy: 0.9998 - val_loss: 1.4700 - val_accuracy: 0.1492
Epoch 3/5
22/22 [==============================] - 1s 66ms/step - loss: 0.0423 - accuracy: 0.9999 - val_loss: 1.5074 - val_accuracy: 0.1474
Epoch 4/5
22/22 [==============================] - 2s 71ms/step - loss: 0.0179 - accuracy: 0.9999 - val_loss: 1.5494 - val_accuracy: 0.1431
Epoch 5/5
22/22 [==============================] - 2s 81ms/step - loss: 0.0095 - accuracy: 0.9999 - val_loss: 1.6003 - val_accuracy: 0.1415
Epoch 1/5
22/22 [==============================] - 3s 141ms/step - loss: 0.0040 - accuracy: 0.9999 - val_loss: 1.7346 - val_accuracy: 0.1411
Epoch 2/5
22/22 [==============================] - 1s 62ms/step - loss: 8.4549e-04 - accuracy: 1.0000 - val_loss: 1.8487 - val_accuracy: 0.1411
Epoch 3/5
22/22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


2017/2017 [==============================] - 14s 7ms/step - loss: 2.6318 - accuracy: 0.8320


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 0s 4ms/step - loss: 9.4989 - accuracy: 0.2500


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


  1/119 [..............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 6ms/step - loss: 10.6234 - accuracy: 0.2500


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[2.6318321228027344, 9.498917579650879], [2.908022403717041, 10.467752456665039], [2.9081966876983643, 10.623394012451172]]
Accuracy for iterations:  [[0.8320106863975525, 0.25], [0.8319796919822693, 0.25], [0.8319796919822693, 0.25]]
F1 for iterations:  [[0.7557758326595134, 0.1], [0.7557013693602648, 0.1], [0.7557013693602648, 0.1]]
Precision for iterations:  [[0.6981405187221714, 0.0625], [0.6922352459557739, 0.0625], [0.6922352459557739, 0.0625]]
Recall for iterations:  [[0.8320106640213281, 0.25], [0.8319796639593279, 0.25], [0.8319796639593279, 0.25]]


### Cat7B

In [14]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7B-Part7.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [15]:

num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [16]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idCAT701.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [17]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT701.hdf5')

Epoch 1/5
2/2 [==============================] - 1s 572ms/step - loss: 1.2653 - accuracy: 0.2374 - val_loss: 1.4474 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 61ms/step - loss: 1.2019 - accuracy: 0.3028 - val_loss: 1.4501 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 58ms/step - loss: 1.1457 - accuracy: 0.4944 - val_loss: 1.4531 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 126ms/step - loss: 1.0957 - accuracy: 0.5635 - val_loss: 1.4563 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 26ms/step - loss: 1.0531 - accuracy: 0.5725 - val_loss: 1.4594 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 1s 302ms/step - loss: 1.0220 - accuracy: 0.5718 - val_loss: 1.4631 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 42ms/step - loss: 0.9857 - accuracy: 0.5890 - val_loss: 1.4662 - val_accuracy: 0.0000e+00
Epo

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


119/119 [==============================] - 1s 5ms/step - loss: 1.3169 - accuracy: 0.4958


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
2/2 [==============================] - 1s 633ms/step - loss: 0.7574 - accuracy: 0.6544 - val_loss: 1.4539 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 50ms/step - loss: 0.7508 - accuracy: 0.6574 - val_loss: 1.4556 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 47ms/step - loss: 0.7485 - accuracy: 0.6589 - val_loss: 1.4576 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 53ms/step - loss: 0.7445 - accuracy: 0.6619 - val_loss: 1.4580 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 60ms/step - loss: 0.7412 - accuracy: 0.6672 - val_loss: 1.4572 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 1s 563ms/step - loss: 0.7298 - accuracy: 0.6657 - val_loss: 1.4529 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 83ms/step - loss: 0.7257 - accuracy: 0.6642 - val_loss: 1.4520 - val_accuracy: 0.0000e+00
Epoc

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


119/119 [==============================] - 1s 9ms/step - loss: 1.2599 - accuracy: 0.5187
Epoch 1/5
2/2 [==============================] - 1s 741ms/step - loss: 0.7138 - accuracy: 0.6747 - val_loss: 1.4519 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 52ms/step - loss: 0.7085 - accuracy: 0.6732 - val_loss: 1.4536 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 68ms/step - loss: 0.7054 - accuracy: 0.6672 - val_loss: 1.4539 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 43ms/step - loss: 0.7031 - accuracy: 0.6762 - val_loss: 1.4542 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 113ms/step - loss: 0.7002 - accuracy: 0.6777 - val_loss: 1.4540 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 2s 792ms/step - loss: 0.6907 - accuracy: 0.6702 - val_loss: 1.4494 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 185m

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


119/119 [==============================] - 1s 7ms/step - loss: 1.1857 - accuracy: 0.5751


In [22]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.3481199741363525, 1.3168654441833496], [1.3654203414916992, 1.2598567008972168], [1.36812162399292, 1.1857095956802368]]
Accuracy for iterations:  [[0.5344410538673401, 0.49578502774238586], [0.2978641092777252, 0.5187038779258728], [0.3018166124820709, 0.5750790238380432]]
F1 for iterations:  [[0.644208241759206, 0.40919421074317874], [0.3872641646744735, 0.44868519200137996], [0.3877987778265896, 0.5446580550194253]]
Precision for iterations:  [[0.8820651862225276, 0.362551262101792], [0.8909300855665555, 0.6353503975044406], [0.892385208165991, 0.6465010605536355]]
Recall for iterations:  [[0.5344410688821377, 0.4957850368809273], [0.29786409572819145, 0.5187038988408852], [0.3018166036332073, 0.5750790305584826]]


### Cat7C

In [23]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat7C-Part7.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [24]:

num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [25]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idCAT702.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [26]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT702.hdf5')

Epoch 1/5
2/2 [==============================] - 1s 279ms/step - loss: 2.0168 - accuracy: 0.0146 - val_loss: 1.4286 - val_accuracy: 0.0034
Epoch 2/5
2/2 [==============================] - 0s 66ms/step - loss: 1.9238 - accuracy: 0.0232 - val_loss: 1.4226 - val_accuracy: 0.0103
Epoch 3/5
2/2 [==============================] - 0s 74ms/step - loss: 1.8365 - accuracy: 0.0378 - val_loss: 1.4168 - val_accuracy: 0.0137
Epoch 4/5
2/2 [==============================] - 0s 77ms/step - loss: 1.7538 - accuracy: 0.0696 - val_loss: 1.4112 - val_accuracy: 0.0137
Epoch 5/5
2/2 [==============================] - 0s 71ms/step - loss: 1.6764 - accuracy: 0.0988 - val_loss: 1.4059 - val_accuracy: 0.0205
Epoch 1/5
2/2 [==============================] - 1s 741ms/step - loss: 1.6189 - accuracy: 0.1391 - val_loss: 1.3701 - val_accuracy: 0.5332
Epoch 2/5
2/2 [==============================] - 0s 59ms/step - loss: 1.5554 - accuracy: 0.1568 - val_loss: 1.3687 - val_accuracy: 0.6545
Epoch 3/5
2/2 [=================

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


  1/119 [..............................] - ETA: 0s - loss: 0.8723 - accuracy: 1.0000WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0104s). Check your callbacks.


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 0s 3ms/step - loss: 1.3039 - accuracy: 0.4926


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


2017/2017 [==============================] - 8s 4ms/step - loss: 0.8514 - accuracy: 0.9115


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 0s 2ms/step - loss: 1.8647 - accuracy: 0.5045


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0600913763046265, 1.3038921356201172], [0.8453078866004944, 1.4476186037063599], [0.8513851165771484, 1.864727258682251]]
Accuracy for iterations:  [[0.8977618217468262, 0.49262380599975586], [0.9097743034362793, 0.49394097924232483], [0.9115258455276489, 0.504478394985199]]
F1 for iterations:  [[0.8803900154319717, 0.340209746720258], [0.8909246515695796, 0.3432699342301511], [0.8959708103565615, 0.36796612103987975]]
Precision for iterations:  [[0.8775480551662516, 0.46375927497806435], [0.9290637761628321, 0.418975261728751], [0.9315069742871185, 0.43966165413533825]]
Recall for iterations:  [[0.897761795523591, 0.49262381454162274], [0.9097743195486391, 0.493940990516333], [0.9115258230516461, 0.5044783983140148]]
